# LoRA Fine-Tuning: DEEP Dataset (Düzeltilmiş)

**Önemli**: Runtime > Change runtime type > **T4 GPU** seçin!

## 1. GPU Kontrolü

In [ ]:
!nvidia-smi

## 2. Paketleri Kur

In [ ]:
!pip install -q torch transformers peft datasets accelerate bitsandbytes tqdm

## 3. Projeyi İndir

In [ ]:
# Varsa sil
!rm -rf /content/Lora-fine-tune

# Tekrar indir
!git clone https://github.com/B0DH1i/Lora-fine-tune.git
%cd /content/Lora-fine-tune

# __init__.py dosyalarını oluştur
!touch config/__init__.py models/__init__.py data/__init__.py training/__init__.py evaluation/__init__.py scripts/__init__.py

print("✓ Proje hazır")

## 4. Google Drive Bağla

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
checkpoint_dir = '/content/drive/MyDrive/lora_checkpoints/deep'
os.makedirs(checkpoint_dir, exist_ok=True)

print(f"✓ Checkpoint dizini: {checkpoint_dir}")

## 5. Config Yükle (8-bit + Düşük Memory)

In [ ]:
import sys
import os

os.chdir('/content/Lora-fine-tune')
sys.path.insert(0, '/content/Lora-fine-tune')

# Config'leri yükle
with open('config/training_config.py', 'r') as f:
    exec(f.read(), globals())
with open('config/model_config.py', 'r') as f:
    exec(f.read(), globals())

# DÜŞÜK MEMORY AYARLARI
TrainingConfig.use_flash_attention_2 = False
TrainingConfig.gradient_checkpointing = True
TrainingConfig.per_device_batch_size = 1
TrainingConfig.gradient_accumulation_steps = 32  # Yüksek (memory için)
TrainingConfig.max_length_solution = 512  # Düşük (1024'ten)
TrainingConfig.use_8bit = True  # 8-bit aktif

print("✓ Config hazır (8-bit + düşük memory)")
print(f"  Learning rate: {TrainingConfig.learning_rate}")
print(f"  LoRA rank: {ModelConfig.lora_r}")
print(f"  Context length: {TrainingConfig.max_length_solution}")
print(f"  Batch size: {TrainingConfig.per_device_batch_size}")
print(f"  Gradient accumulation: {TrainingConfig.gradient_accumulation_steps}")

## 6. Model Yükle (8-bit)

In [ ]:
# Model loader'ı yükle
with open('models/model_loader.py', 'r') as f:
    exec(f.read(), globals())

print("Model yükleniyor (8-bit)...")
model, tokenizer = load_model_and_tokenizer(
    use_flash_attention=False,
    load_in_8bit=True  # 8-BIT!
)
print("✓ Model yüklendi (8-bit)")

## 7. LoRA Setup

In [ ]:
# LoRA setup'ı yükle
with open('models/lora_setup.py', 'r') as f:
    exec(f.read(), globals())

print("LoRA yapılandırılıyor...")
model = setup_lora(model, use_8bit=True)
print("✓ LoRA yapılandırıldı")

## 8. Dataset Yükle

In [ ]:
from datasets import load_dataset

print("DEEP dataset yükleniyor...")

# Dataset yükle
dataset = load_dataset("Naholav/CodeGen-Deep-5K")

# Train/test split
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)

print(f"✓ Dataset split - Train: {len(dataset['train'])}, Test: {len(dataset['test'])}")

# Preprocessing
def preprocess_function(examples):
    system_prompt = "You are an expert Python programmer. Please read the problem carefully before writing any Python code."
    prompt = f"{system_prompt}\n\nProblem:\n{examples['input']}\n\nSolution:\n{examples['solution']}"
    
    tokenized = tokenizer(
        prompt,
        truncation=True,
        max_length=512,  # Düşük memory için
        padding="max_length",
        return_tensors=None
    )
    
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

print("Preprocessing yapılıyor...")
train_dataset = dataset["train"].map(
    preprocess_function,
    remove_columns=dataset["train"].column_names,
    desc="Train preprocessing"
)

eval_dataset = dataset["test"].map(
    preprocess_function,
    remove_columns=dataset["test"].column_names,
    desc="Test preprocessing"
)

print(f"✓ Dataset hazır - Train: {len(train_dataset)}, Eval: {len(eval_dataset)}")

## 9. Trainer Setup

In [ ]:
# Trainer ve callbacks yükle
with open('training/trainer.py', 'r') as f:
    exec(f.read(), globals())
with open('training/callbacks.py', 'r') as f:
    exec(f.read(), globals())

print("Trainer yapılandırılıyor...")
trainer = setup_trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    output_dir=checkpoint_dir,
    run_name="deep_training_8bit"
)

print("✓ Trainer hazır")

## 10. Training Başlat

In [ ]:
print("\n" + "="*60)
print("TRAINING BAŞLIYOR!")
print("="*60)
print("\nAyarlar:")
print(f"  - 8-bit quantization: Aktif")
print(f"  - Context length: 512 token")
print(f"  - Batch size: 1")
print(f"  - Gradient accumulation: 32")
print(f"  - Effective batch size: 32")
print("\nTahmini süre: 3-5 saat")
print("Colab oturumunu açık tutun!\n")

# Training başlat
trainer.train()

## 11. Model Kaydet

In [ ]:
import os

final_model_path = os.path.join(checkpoint_dir, "final_model")
print(f"Final model kaydediliyor: {final_model_path}")

trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path)

print("\n" + "="*60)
print("✓ TRAINING TAMAMLANDI!")
print("="*60)
print(f"\nModel: {final_model_path}")
print(f"Log'lar: {os.path.join(checkpoint_dir, 'logs')}")

## 12. Log'ları İndir

In [ ]:
# Log'ları zip'le
!zip -r deep_training_logs.zip {checkpoint_dir}/logs

# İndir
from google.colab import files
files.download('deep_training_logs.zip')

print("✓ Log'lar indirildi")